<a href="https://colab.research.google.com/github/Sunday-Okey/IVA_Research_Works/blob/master/IVA_S_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy  as np
import scipy as sc
from sklearn.covariance import GraphicalLassoCV

import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
import timeit # for timing functions
import time # for getting time related data
import numpy as np
import scipy.io as sio
from sklearn.svm import SVC
import pandas as pd
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold
import scipy.stats

import scipy as sc
import scipy.linalg as linalg
from scipy.special import gamma


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

import os

# Set your working directory to a folder in your Google Drive. This way, if your notebook times out,
# your files will be saved in your Google Drive!

# the base Google Drive directory
root_dir = "/content/drive/My Drive/"

# choose where you want your project files to be saved
project_folder = "Colab Notebooks/Computer_Vision/"

def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if not os.path.isdir(root_dir + project_folder):
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(root_dir + project_folder)

  # create a test file to make sure it shows up in the right place
  !touch 'new_file_in_working_directory.txt'
  print('\nYour working directory was changed to ' + root_dir + project_folder + \
        "\n\nAn empty text file was created there. You can also run !pwd to confirm the current working directory." )

create_and_set_working_directory(project_folder)



Your working directory was changed to /content/drive/My Drive/Colab Notebooks/Computer_Vision/

An empty text file was created there. You can also run !pwd to confirm the current working directory.


In [ ]:
pkl_file_path = 'test_w_time_10232023.pkl'

with open(pkl_file_path, 'rb') as file:
    context = pickle.load(file)

context_df_test = pd.DataFrame(context)

In [ ]:
# pip install "pandas<2.0.0"

In [ ]:
pkl_file_path = 'train_w_time_10232023.pkl'

with open(pkl_file_path, 'rb') as file:
    context = pickle.load(file)

context_df_train = pd.DataFrame(context)

In [ ]:
# Assuming context_df and context_df_test are already defined Pandas DataFrames
concatenated_df = pd.concat([context_df_train, context_df_test], ignore_index=True)
concatenated_df.columns

Index(['tweet_id', 'lang', 'res_image_vectors', 'vgg_image_vectors',
       'tweet_text', 'label', 'event', 'new_img_id', 'clean_tweet', 'cl_len',
       'text_feats_1', 'text_feats_2', 'pooler', 'sm_last_four_concat',
       'mn_last_four_concat', 'mn_2ndtolast_lyr', 'sm_2ndtolast_lyr',
       'num_friends', 'num_followers', 'folfriend_ratio', 'times_listed',
       'has_url', 'is_verified', 'num_posts', 'num_words', 'text_length',
       'contains_questmark', 'num_questmark', 'contains_exclammark',
       'num_exclammark', 'contains_happyemo', 'contains_sademo',
       'contains_firstorderpron', 'contains_secondorderpron',
       'contains_thirdorderpron', 'num_uppercasechars', 'num_possentiwords',
       'num_negsentiwords', 'num_mentions', 'num_hashtags', 'num_URLs',
       'num_retweets', 'semi_clean', 'reading_ease', 'reading_grade',
       'compound_score', 'neg_score', 'pos_score', 'neutral_score',
       'vgg19_img_vectors', 'Topic20', 'Topic10', 'Topic5', 'caption',
       'c

In [ ]:
with pd.option_context('display.max_colwidth', None):
  selected_columns = ['tweet_text', 'clean_tweet']
  print(concatenated_df[selected_columns].iloc[0])

tweet_text     Don't need feds to solve the #bostonbombing when we have #4chan!! http://t.co/eXQTPZqqbG
clean_tweet                                              [don, need, feds, solve, bostonbombing, 4chan]
Name: 0, dtype: object


In [ ]:
len(concatenated_df['mdy'].unique())

386

In [ ]:
concatenated_df.shape

(9938, 67)

In [ ]:
concatenated_df['event'].unique()

array(['boston', 'malaysia', 'passport', 'sandy', 'sochi', 'bringback',
       'columbianChemicals', 'elephant', 'livr', 'pigFish', 'underwater',
       'eclipse', 'samurai', 'nepal', 'garissa', 'fuji_lenticular',
       'bush_book', 'gandhi_dancing', 'attacks_paris', 'bowie_david',
       'pakistan_explosion', 'refugees', 'protest', 'not_afraid',
       'rio_moon', 'immigrants', 'john_guevara', 'syrian_children',
       'mc_donalds', 'brussels_explosions', 'half_everything',
       'hubble_telescope', 'woman_14', 'north_korea', 'five_headed',
       'burst_kfc', 'black_lion', 'nazi_submarine'], dtype=object)

In [ ]:
time_text = concatenated_df.groupby('mdy')
time_counts = time_text['tweet_text'].count()
print(time_counts.max())
print(time_counts.min())
time_counts.sum()

2324
1


9938

In [ ]:
unique_events_per_date = time_text['event'].unique()

In [ ]:
unique_events_df = unique_events_per_date.reset_index(name='unique_events')
unique_events_df.head()

,mdy,unique_events
0,2011-10-04,[five_headed]
1,2011-10-22,[bush_book]
2,2011-11-14,[five_headed]
3,2011-12-21,[burst_kfc]
4,2012-01-21,[five_headed]


In [ ]:
print(time_counts)

mdy
2011-10-04     1
2011-10-22     1
2011-11-14     1
2011-12-21     1
2012-01-21     1
              ..
2016-03-25     2
2016-03-26     1
2016-03-27     8
2016-03-28    26
2016-06-06     1
Name: tweet_text, Length: 386, dtype: int64


In [ ]:
from helpers_iva import normalize_column_vectors, decouple_trick, bss_isi, whiten_data
from initializations import jbss_sos, cca

In [ ]:
event_text = concatenated_df.groupby('event')
event_counts = event_text['event'].count()
event_counts

event
attacks_paris           181
black_lion                7
boston                  474
bowie_david              64
bringback               109
brussels_explosions       9
burst_kfc                15
bush_book                25
columbianChemicals      182
eclipse                 200
elephant                 11
five_headed               5
fuji_lenticular         147
gandhi_dancing           13
garissa                  44
half_everything          34
hubble_telescope         17
immigrants               30
john_guevara             10
livr                      2
malaysia                165
mc_donalds                5
nazi_submarine           10
nepal                  1250
north_korea               7
not_afraid               48
pakistan_explosion       34
passport                 41
pigFish                  12
protest                  34
refugees                 53
rio_moon                 26
samurai                 213
sandy                  6073
sochi                   259
syrian_childre

In [ ]:
from datetime import datetime
def convert_datetime(date_time):

    formatted_date = date_time.strftime('%y%m%d')

    return formatted_date

In [ ]:
text_time_t = {}
vocab_time_t = {}
total_vocab = []
all_unique_elements = set()

# Iterate through the grouped data
time_t = 0
for time_stamp, group in event_text:
  all_elements = []
  unique_elements = set()
  for elements in group['clean_tweet']:
    all_elements.extend(elements)
    unique_elements.update(elements)
  text_time_t[time_t] = all_elements
  vocab_time_t[time_t] = list(unique_elements)
  all_unique_elements.update(unique_elements)
  time_t += 1
  # print("text_time_t", len(all_elements))
  # print("vocab_time_t", len(list(unique_elements)))

total_vocab = list(all_unique_elements)

In [ ]:
print("text_time_t", list(text_time_t))
print("vocab_time_t", list(vocab_time_t))

text_time_t [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]
vocab_time_t [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]


In [ ]:
for t in text_time_t:
    print(len(vocab_time_t[t]), len(text_time_t[t]))

254 1585
17 51
962 4039
184 494
233 533
32 89
27 85
130 290
407 1441
325 1579
6 28
18 32
270 1177
64 113
177 459
37 301
45 149
167 270
18 67
16 20
426 1365
27 49
15 81
1060 11120
18 61
147 268
71 289
93 338
62 80
145 270
217 525
57 237
108 2060
4963 42637
800 2043
34 120
106 856
29 101


In [ ]:
len(total_vocab)

7791

In [ ]:
co_oc_file_path = 'combined_co_occurence.pkl'

with open(co_oc_file_path, 'rb') as file:
    loaded_co_oc = pickle.load(file)

In [ ]:
len(total_vocab)

7791

In [ ]:
mylist = []
for co_oc in loaded_co_oc:
  if co_oc.shape[0] >= 167:
    mylist.append(co_oc)
    print(co_oc.shape)

(254, 7791)
(962, 7791)
(184, 7791)
(233, 7791)
(407, 7791)
(325, 7791)
(270, 7791)
(177, 7791)
(167, 7791)
(426, 7791)
(1060, 7791)
(217, 7791)
(4963, 7791)
(800, 7791)


In [ ]:
from sklearn.decomposition import PCA
import numpy as np

# Generate random data matrix X
np.random.seed(42)  # for reproducibility
X_1 = np.random.rand(7791, 254) #7791 is the whole vocabulary and 254 is the vocabulary for event 1

W_1 = np.random.rand(130, 130) # This is the matrix that we obtain from IVA

# Fit PCA to your data
pca = PCA(n_components=130) #130 is the PCA order
mapped1 = pca.fit_transform(X_1) # shape num_samples x num_components

print(mapped1.shape)

F1 = np.dot(np.linalg.pinv(X_1), mapped1)

print(F1.shape)

A_1 = np.dot(np.linalg.pinv(F1.T), np.linalg.inv(W_1)) # shape (num features = len reliable vocab) X (num components = N)

print(A_1.shape) #each row will be a word embedding for event 1.

# A_2 will have a different size. It will be 962 x 130.
# A_3 will be 182 x 130. and so on

(7791, 130)
(254, 130)
(254, 130)


In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler

# import packages for modeling
from sklearn.decomposition import IncrementalPCA
from sklearn.decomposition import PCA

data = []
#instantiate scaler on training data
scl_txt = StandardScaler(with_mean = True, with_std = False)
pca_txt_mod = PCA(n_components = 167, svd_solver='full')
for item in mylist:
    X_scaled = scl_txt.fit_transform(item.T)
    # Fit and transform text data
    X = pca_txt_mod.fit_transform(X_scaled )
    # mylist.append(X)
    print(X.shape)
    data.append(X )



(7791, 167)
(7791, 167)
(7791, 167)
(7791, 167)
(7791, 167)
(7791, 167)
(7791, 167)
(7791, 167)
(7791, 167)
(7791, 167)
(7791, 167)
(7791, 167)
(7791, 167)
(7791, 167)


In [ ]:
mydata = [item.T for item in data]
for item in mydata:
  print(item.shape)

(167, 7791)
(167, 7791)
(167, 7791)
(167, 7791)
(167, 7791)
(167, 7791)
(167, 7791)
(167, 7791)
(167, 7791)
(167, 7791)
(167, 7791)
(167, 7791)
(167, 7791)
(167, 7791)


In [ ]:
Z = np.stack(mydata, axis = 2)

In [ ]:
Z.shape

(167, 7791, 14)

In [ ]:
# def iva_spice(X, whiten=True, verbose=False, A=None, max_iter=512, W_diff_stop=1e-6, alpha0=1.0):

#     """
#     Implementation of all the second-order (Gaussian) independent vector analysis (IVA) algorithms.
#     Real valued using gradient optimization.

#     For a general description of the algorithm and its relationship with others,
#     see https://urldefense.com/v3/__http://mlsp.umbc.edu/jointBSS_introduction.html__;!!IaT_gp1N!gcLMVNHaocM9dxMD8uC66Ees6BVgmALY_-uO7NJnnjvczOwdkhMkEEEWmdbo8Yqt2_o$


#     Parameters
#     ----------
#     X : np.ndarray
#         data matrix of dimensions N x T x K.
#         Data observations are from K data sets, i.e., X[k] = A[k] @ S[k], where A[k] is an N x N
#         unknown invertible mixing matrix, and S[k] is N x T matrix with the nth row corresponding to
#         T samples of the nth source in the kth dataset. This enforces the assumption of an equal
#         number of samples in each dataset.
#         For IVA, it is assumed that a source is statistically independent of all the other sources
#         within the dataset and exactly dependent on at most one source in each of the other
#         datasets.

#     whiten : bool, optional
#         if True, data is whitened.

#     verbose : bool, optional
#         enables print statements if True. Automatically set to True if A is provided

#     A : np.ndarray, optional
#         true mixing matrices A of dimensions N x N x K, automatically sets verbose to True

#     max_iter : int, optional
#         max # of iterations

#     W_diff_stop : float, optional
#         stopping criterion

#     alpha0 : float, optional
#         initial step size scaling


#     Returns
#     -------
#     W : np.ndarray
#         the estimated demixing matrices of dimensions N x N x K so that ideally
#         W[k] @ A[k] = P @ D[k], where P is any arbitrary permutation matrix and D[k] is any
#         diagonal invertible (scaling) matrix. Note that P is common to all datasets. This is
#         to indicate that the local permutation ambiguity between dependent sources
#         across datasets should ideally be resolved by IVA.

#     cost : float
#         the cost for each iteration

#     Sigma_N : np.ndarray
#         Covariance matrix of each source component vector, with dimensions K x K x N

#     isi : float
#         joint isi (inter-symbol-interference) for each iteration. Only available if user
#         supplies true mixing matrices for computing a performance metric. Else returns np.nan.


#     # TODO: Notes, References

#     """

#     # throw errors for incorrect X argument
#     if X.ndim != 3:
#         raise AssertionError('X must have dimensions N x T x K.')
#     elif X.shape[2] == 1:
#         raise AssertionError('There must be ast least K=2 datasets.')

#     # get dimensions
#     N, T, K = X.shape

#     # determine if (correct) A argument provided
#     if A is not None:
#         supply_A = True
#         if A.shape[0] != N or A.shape[1] != N or A.shape[2] != K:
#             raise AssertionError('A must have dimensions N x N x K.')
#     else:
#         supply_A = False

#     blowup = 1e3
#     alpha_scale = 0.9   # alpha0 to alpha0*alpha_scale when cost does not decrease
#     alpha_min = W_diff_stop  # can also be chosen different

#     if whiten:
#         X, V, _ = whiten_data(X)

#     # calculate cross-covariance matrices of X
#     R_xx = np.zeros((N, N, K, K), dtype=X.dtype)
#     for k1 in range(K):
#         for k2 in range(k1, K):
#             R_xx[:, :, k1, k2] = 1 / T * X[:, :, k1] @ np.conj(X[:, :, k2].T)
#             R_xx[:, :, k2, k1] = np.conj(R_xx[:, :, k1, k2].T)  # R_xx is Hermitian

#     # Initializations

#     # Initialize W through random initialization
#     W = np.random.randn(N, N, K)
#     Y = np.zeros((N, T, K))

#     for k in range(K):
#         W[:, :, k] = np.linalg.solve(sc.linalg.sqrtm(W[:, :, k] @ W[:, :, k].T), W[:, :, k])

#     if supply_A:
#         # only reason to supply A matrices is to display running performance and to compute output_isi
#         verbose = True
#         isi = np.zeros(max_iter)
#         if whiten:
#             # A matrix is conditioned by V if data is whitened
#             A_w = np.copy(A)
#             for k in range(K):
#                 A_w[:, :, k] = V[:, :, k] @ A_w[:, :, k]
#         else:
#             A_w = np.copy(A)
#     else:
#         isi = np.nan

#     # Check rank of data-covariance matrix: should be full rank, if not we inflate (this is ad hoc)
#     # concatenate all covariance matrices in a big matrix
#     R_xx_all = np.moveaxis(R_xx, [0, 1, 2, 3], [0, 2, 1, 3]).reshape(
#         (R_xx.shape[0] * R_xx.shape[2], R_xx.shape[1] * R_xx.shape[3]), order='F')
#     rank = np.linalg.matrix_rank(R_xx_all)
#     if rank < (N * K):
#         # inflate Rx
#         _, k, _ = np.linalg.svd(R_xx_all)
#         R_xx_all += k[rank - 1] * np.eye(N * K)  # add smallest singular value to main diagonal
#         R_xx = np.moveaxis(
#             R_xx_all.reshape(R_xx.shape[0], R_xx.shape[2], R_xx.shape[1], R_xx.shape[3], order='F'),
#             [0, 2, 1, 3], [0, 1, 2, 3])

#     # Initialize some local variables
#     cost = np.zeros(max_iter)
#     cost_const = K * np.log(2 * np.pi * np.exp(1))  # local constant

#     grad = np.zeros((N, K))

#     # Main Iteration Loop
#     for iteration in range(max_iter):
#         term_criterion = 0

#         # Some additional computations of performance via ISI when true A is supplied
#         if supply_A:
#             avg_isi, joint_isi = bss_isi(W, A_w)
#             isi[iteration] = joint_isi

#         W_old = np.copy(W)
#         cost[iteration] = 0
#         for k in range(K):
#             cost[iteration] -= np.log(np.abs(np.linalg.det(W[:, :, k])))
#             # Min. cost function

#         Q = 0
#         R = 0

#         # Calculate SCV (new code)
#         for k in range(K):
#             Y[:,:,k] = np.matmul(W[:,:,k], X[:,:,k])

#         # Loop over each SCV
#         for n in range(N):
#             Wn = np.conj(W[n, :, :]).flatten(order='F')

#             Sigma_n = np.eye(K)

#             # Graphical Lasso - SPICE (new code)
#             model = GraphicalLassoCV(tol=1e-2, max_iter=200) # max_iterations increase
#             model.fit(Y[n, :, :])
#             Sigma_n = model.covariance_
#             Sigma_inv = model.precision_

#             cost[iteration] += 0.5 * (cost_const + np.log(np.abs(np.linalg.det(Sigma_n))))

#             # Decoupling trick
#             hnk, Q, R = decouple_trick(W, n, Q, R)

#             for k in range(K):
#                 # Analytic derivative of cost function with respect to vn
#                 # Code below is efficient implementation of computing the gradient, which is
#                 # independent of T
#                 grad[:, k] = - hnk[:, k] / (W[n, :, k] @ hnk[:, k])


#                 for kk in range(K):
#                     grad[:, k] += R_xx[:, :, k, kk] @ np.conj(W[n, :, kk]) * Sigma_inv[kk, k]

#                 wnk = np.conj(W[n, :, k])

#                 grad_norm = normalize_column_vectors(grad[:, k])
#                 grad_norm_proj = normalize_column_vectors(grad_norm - np.conj(
#                     wnk) @ grad_norm * wnk)  # non-colinear direction normalized
#                 W[n, :, k] = np.conj(
#                     normalize_column_vectors(wnk - alpha0 * grad_norm_proj))

#         for k in range(K):
#             # in original matlab code, this is the term criterion
#             term_criterion = np.maximum(term_criterion, np.amax(
#                 1 - np.abs(np.diag(W_old[:, :, k] @ np.conj(W[:, :, k].T)))))

#         # Decrease step size alpha if cost increased from last iteration
#         if iteration > 0:
#             if cost[iteration] > cost[iteration - 1]:
#                 alpha0 = np.maximum(alpha_min, alpha_scale * alpha0)

#         # Check the termination condition
#         if term_criterion < W_diff_stop or iteration == max_iter - 1:
#             break
#         elif term_criterion > blowup or np.isnan(cost[iteration]):
#             for k in range(K):
#                 W[:, :, k] = np.eye(N) + 0.1 * np.random.randn(N, N)
#             if verbose:
#                 print('W blowup, restart with new initial value.')

#         # Display Iteration Information
#         if verbose:
#             if supply_A:
#                 print(
#                     f'Step {iteration}: W change: {term_criterion}, Cost: {cost[iteration]}, '
#                     f'Avg ISI: {avg_isi}, Joint ISI: {joint_isi}')
#             else:
#                 print(f'Step {iteration}: W change: {term_criterion}, Cost: {cost[iteration]}')

#     # Finish Display
#     if iteration == 0 and verbose:
#         if supply_A:
#             print(
#                 f'Step {iteration}: W change: {term_criterion}, Cost: {cost[iteration]}, '
#                 f'Avg ISI: {avg_isi}, Joint ISI: {joint_isi}')
#         else:
#             print(f'Step {iteration}: W change: {term_criterion}, Cost: {cost[iteration]}')

#     # Clean-up Outputs
#     cost = cost[0:iteration + 1]

#     if supply_A:
#         isi = isi[0:iteration + 1]

#     if whiten:
#         for k in range(K):
#             W[:, :, k] = W[:, :, k] @ V[:, :, k]
#     else:  # no prewhitening
#         # Scale demixing vectors to generate unit variance sources
#         for n in range(N):
#             for k in range(K):
#                 W[n, :, k] /= np.sqrt(W[n, :, k] @ R_xx[:, :, k, k] @ np.conj(W[n, :, k]))

#     # Resort order of SCVs: Order the components from most to least ill-conditioned

#     # First, compute the data covariance matrices (by undoing any whitening)
#     if whiten:
#         for k1 in range(K):
#             for k2 in range(k1, K):
#                 R_xx[:, :, k1, k2] = np.linalg.solve(V[:, :, k2],
#                                                      np.linalg.solve(V[:, :, k1],
#                                                                      R_xx[:, :, k1, k2]).T).T
#                 R_xx[:, :, k2, k1] = np.conj(R_xx[:, :, k1, k2].T)  # R_xx is Hermitian

#     # Second, compute the determinant of the SCVs
#     detSCV = np.zeros(N)
#     Sigma_N = np.zeros((K, K, N))

#     for k in range(K):
#         Y[:,:,k] = np.matmul(W[:,:,k], X[:,:,k])

#     for n in range(N):

#         Sigma_n = np.zeros((K, K))

#         # Graphical Lasso - SPICE (new code)
#         model = GraphicalLassoCV(tol=1e-2, max_iter=200) # max_iterations increase
#         model.fit(Y[n, :, :])
#         Sigma_n = model.covariance_
#         Sigma_inv = model.precision_

#         Sigma_N[:, :, n] = Sigma_n

#         detSCV[n] = np.linalg.det(Sigma_n)

#     # Third, sort and apply
#     isort = np.argsort(detSCV)
#     Sigma_N = Sigma_N[:, :, isort]
#     for k in range(K):
#         W[:, :, k] = W[isort, :, k]

#     return W, cost, Sigma_N, isi


In [ ]:
# # Run IVA_SPICE model on data to get W matrices
# import time
# start_time = time.time() # collects start time

# W, c, sig, isi = iva_spice(Z, whiten=False, verbose=True)

# print(time.time() - start_time)

In [ ]:
pkl_file_path = 'IVA_S_14_output.pkl'

with open(pkl_file_path, 'rb') as file:
    context = pickle.load(file)

iva_spice = pd.Series(context)
iva_spice["W"].shape


W = iva_spice["W"]
W.shape

(167, 167, 14)

In [ ]:
len(W)

167

In [ ]:
# # Initialize an empty list to store the split matrices
Ws = []


for i in range(14):
    W_split = np.squeeze(np.split(W, 14, axis=2)[i], axis=2)
    Ws.append(W_split)

# Ws now contains W1, W2, ...,
W1, W2, W3, W4, W5, W6, W7, W8, W9, W10, W11, W12, W13, W14 = Ws


In [ ]:
W.shape

(167, 167, 14)

In [ ]:
mydata[0].shape

(167, 7791)

In [ ]:
len(mydata)

14

In [ ]:
# Generate S1 and S2
Ws = [W1, W2, W3, W4, W5, W6, W7, W8, W9, W10, W11, W12, W13, W14]
Ss = []
# S1_tr = W1 @ np.transpose(X) #Text Components

for W, val in zip(Ws, mydata):
  S_tr = W @ (val)
  Ss.append(S_tr)

In [ ]:
# Data Fusion - Averaging
df_IVA_M_EMK = 0
for i in Ss:
  df_IVA_M_EMK += i.T
df_IVA_M_EMK = df_IVA_M_EMK / 14

In [ ]:
df_IVA_M_EMK = np.array(df_IVA_M_EMK)

In [ ]:
def embedding_feats(list_of_lists):
    DIMENSION = 167
    feats = []
    for tokens in list_of_lists:
        feat_for_this = np.zeros(DIMENSION)
        count_for_this = 0
        for token in tokens:
            if token in total_vocab: # Check if the token is in the model's vocabulary
                index = total_vocab.index(token)
                feat_for_this += df_IVA_M_EMK.T[:, index]  # Access the vector for the token

        if count_for_this != 0:
            feat_for_this = feat_for_this / count_for_this  # Average the vectors
        feats.append(feat_for_this)
    return feats

train_vectors = embedding_feats(list(context_df_train['clean_tweet']))
test_vectors = embedding_feats(list(context_df_test['clean_tweet']))


In [ ]:
# create function to recode label field
def recode_annotation(txt_field):
    if txt_field == "fake":
        return 1
    else:
        return 0

# run function to recode label field.
train_labels = context_df_train['label'].apply(recode_annotation)
test_labels = context_df_test['label'].apply(recode_annotation)

In [ ]:
# SVC hyperparams to optimize

parameters = {'C':[0.1, 1, 10, 100, 1000],
                'kernel':['rbf']}

scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score, average = 'binary'),
           'recall': make_scorer(recall_score, average = 'binary'),
           'f1': make_scorer(f1_score, average = 'binary')}

In [ ]:
import joblib
np.random.seed(1)
# CV
# cv = StratifiedKFold(n_splits=5, shuffle=True)
svm = SVC(gamma='scale', random_state=1)
svm_GS = GridSearchCV(svm, param_grid = parameters, scoring = scoring, refit='f1', cv=5, return_train_score=True)
svm_GS.fit(train_vectors,train_labels)

# The best estimator found by GridSearchCV
best_model = svm_GS.best_estimator_
best_model

# Save the best model to disk
# joblib.dump(best_model, 'best_svm_model_new_spice.joblib')

SVC(C=0.1, random_state=1)

In [ ]:
svm_GS.best_params_

{'C': 0.1, 'kernel': 'rbf'}

In [ ]:
 # Predict using the best model
predictions = best_model.predict(test_vectors)

In [ ]:
# Calculate metrics
precision = precision_score(test_labels, predictions)
recall = recall_score(test_labels, predictions)
accuracy = accuracy_score(test_labels, predictions)
f1score = f1_score(test_labels, predictions)
# Print the metrics
print(f'Precision: {precision * 100}')
print(f'Recall: {recall * 100}')
print(f'Accuracy: {accuracy * 100}')
print(f'f1_score: {f1score * 100}')

df = pd.DataFrame({'Metrics': ['Accuracy','Precision', 'Recall', 'f1_score'], 'Performance Results': [accuracy,precision, recall,f1score]})
print('\n')
df

Precision: 58.66834170854271
Recall: 100.0
Accuracy: 58.66834170854271
f1_score: 73.95091053048299




,Metrics,Performance Results
0,Accuracy,0.586683
1,Precision,0.586683
2,Recall,1.000000
3,f1_score,0.739509
